In [ ]:
from kafka import KafkaConsumer

In [ ]:
def stream_stats():
    consumer = KafkaConsumer('wiki_event_stats', bootstrap_servers='kafka:9092')
    for message in consumer:
        # fetch namespace and count of events in that namespace
        wiki_ns = message.key.decode()
        events_cnt = int.from_bytes(message.value, 'big')
        yield wiki_ns, events_cnt

In [ ]:
%matplotlib notebook

import time

import numpy as np
import matplotlib.pyplot as plt

data = []
names = []
width=0.3
plt.rcParams["figure.figsize"]=9,6

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
fig.tight_layout()
fig.show()
fig.canvas.draw()

last_updated = time.time()
for wiki_ns, events_cnt in stream_stats():
    # update in-memory caches for rendering plot
    if wiki_ns not in names:
        names.append(wiki_ns)
        data.append(events_cnt)
    else:
        ns_idx = names.index(wiki_ns)
        data[ns_idx] = events_cnt
    
    # decide to render or not
    if time.time() - last_updated < 1:
        continue
    else:
        last_updated = time.time()
    
    # do the render magic
    ax = plt.subplot(111)
    ax.clear()
    bins = map(lambda x: x-width/2, range(1, len(data)+1))
    ax.bar(list(bins), data, width=width)
    ax.set_xticks(list(map(lambda x: x, range(1, len(data)+1))))
    ax.set_xticklabels(names, rotation=45, rotation_mode="anchor", ha="right")
    fig.tight_layout()
    fig.canvas.draw()